In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import illustris_python as il
import pandas as pd

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential"]
        }
indices = [1893, 2164, 3043, 3229, 3403, 3605, 3760] #np.genfromtxt("./data/tng-100-3/cutdata/early_type_indices.csv")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists= [stars, gas, dm]
group_cat = pd.DataFrame({"id": indices})


In [2]:
def radius(df):
    x = np.array(df["x"])
    y = np.array(df["y"])
    z = np.array(df["z"])
    r = (x**2 + y**2 + z**2)**(1/2)
    return r

In [3]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"])


subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [4]:
#calculate radius
center_of_mass = np.zeros([N,3])
for i in range(N):
    #find center of halo
    min_pot = 0
    for particle in particle_lists:
        #Find particle with lowest potential
        min_pot_value = particle[i]["Potential"].min()
        if min_pot_value < min_pot:
            min_pot = min_pot_value
            #Get particle position and assign as center of galaxy
            min_pot_pos = particle[i][particle[i]["Potential"] == min_pot_value]["Coordinates"].values[0]
    #Save to group catalogue
    center_of_mass[i] = min_pot_pos
    #Calculate new coordinates
    for particle in particle_lists:
        particle[i]["x"] = particle[i]["X"] - min_pot_pos[0]
        particle[i]["y"] = particle[i]["Y"] - min_pot_pos[1]
        particle[i]["z"] = particle[i]["Z"] - min_pot_pos[2]
        particle[i]["r"] = radius(particle[i])
group_cat["CenterOfMassX"] = center_of_mass[:,0]
group_cat["CenterOfMassY"] = center_of_mass[:,1]
group_cat["CenterOfMassZ"] = center_of_mass[:,2]

In [5]:
#Calculate total mass for each particle type
stellar_mass = np.zeros(N)
gas_mass = np.zeros(N)
dm_mass = np.zeros(N)

for i in range(N):
    stellar_mass[i] = stars[i]["Masses"].sum() #or load from snapshot to save time
    gas_mass[i] = gas[i]["Masses"].sum()
    dm_mass[i] = dm[i]["Masses"].sum()

total_mass = stellar_mass + gas_mass + dm_mass
#save value to catalogue
group_cat["SubhaloMassStellar"] = stellar_mass
group_cat["SubhaloMassGas"] = gas_mass
group_cat["SubhaloMassDM"] = dm_mass
group_cat["SubhaloMass"] = total_mass
#print(stars[1])

In [6]:
halfmass_rad = np.zeros(N)
#Calculate half mass radius

for i in range(N):
    temp = stars[i].copy(deep=True)
    temp = temp.sort_values(by="r")
    temp_mass = 0
    n = temp["count"][0]
    r_max = 40
    r = 0
    for j in range(n):
        if temp_mass < (group_cat["SubhaloMassStellar"][i]/2):
            temp_mass = temp_mass + temp["Masses"][j]
        else:
            m1 = temp["Masses"][j-1]
            m2 = temp["Masses"][j]
            M = m1+m2
            halfmass_rad[i] = (m1*temp["r"][j-1] + m2*temp["r"][j])/M #some uncertainty here, now center of mass
            break  
group_cat["SubhaloHalfmassRadStellar"] = halfmass_rad


In [7]:
#print(group_cat["SubhaloHalfmassRadStellar"])
j = 0
for i in indices: 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][i], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][j])
    j = j +1
#print(df_subhalos["SubhaloMassStellar"][1893])

Group catalogue:  22.51544  Snapshot:  24.917712662221792
Group catalogue:  35.24243  Snapshot:  33.96407061049739
Group catalogue:  27.368404  Snapshot:  31.957937076175117
Group catalogue:  28.622637  Snapshot:  43.473352046316556
Group catalogue:  20.585445  Snapshot:  16.469285075727896
Group catalogue:  15.178419  Snapshot:  18.897330597224407
Group catalogue:  21.49089  Snapshot:  10.059873622989697
